In [13]:
import os
import pickle
import pandas as pd
import numpy as np
from multiprocessing import Pool
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [14]:
# Load data
data_path = "../../data"
X_train = pd.read_csv(os.path.join(data_path, "X_train_pca.csv"))
y_train_org = pd.read_csv(os.path.join(data_path, "y_train_pca.csv"))
X_test = pd.read_csv(os.path.join(data_path, "X_test_pca.csv"))
y_test_org = pd.read_csv(os.path.join(data_path, "y_test_pca.csv"))

In [15]:
traits = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional Stability', 'Openness']
random_state=27

In [16]:
# Create results directory
results_path = "../../results"
specific_results_path = os.path.join("../../results", "knn_classification")
os.makedirs(results_path, exist_ok=True)
os.makedirs(specific_results_path, exist_ok=True)

In [17]:
def calc_roc_auc(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n_classes = np.unique(y_true)
    roc_auc_scores = []
    for label in n_classes:
        # Create binary labels for the current class vs. all other classes
        y_true_class = (y_true == label).astype(int)
        y_pred_class = (y_pred == label).astype(int)
        
        # Calculate ROC AUC for the current class
        roc_auc = roc_auc_score(y_true_class, y_pred_class)
        roc_auc_scores.append(roc_auc)
    return roc_auc_scores

In [18]:
# Define the parameter grid you want to search over
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],      # Weighting scheme for neighbors
    'p': [1, 2],                             # Distance metric (1 for Manhattan distance, 2 for Euclidean distance)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute nearest neighbors
    'leaf_size': [10, 20, 30, 40, 50],       # Leaf size for tree-based algorithms
    'metric': ['minkowski', 'manhattan', 'euclidean', 'chebyshev']  # Distance metric for Minkowski distance
}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Create a grid search object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, scoring="accuracy", cv=3, n_jobs=5)

for trait in traits:
    print(f"Processing {trait}")
    trait_bin = trait + "_bin"
    label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
    y_train = [label_mapping[label] for label in y_train_org[trait_bin]]
    y_test = [label_mapping[label] for label in y_test_org[trait_bin]]
    # Fit the grid search to your data
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters and the corresponding score
    print("Best Hyperparameters: ", grid_search.best_params_)
    print("Best Score: ", grid_search.best_score_)

    # Get the best model from the grid search
    best_knn = grid_search.best_estimator_

    # Now, you can use the best_rf model for predictions on your test data
    y_pred = best_knn.predict(X_test)
    y_true = y_test

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    roc_auc = calc_roc_auc(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    print(f'ROC AUC: {roc_auc}')
    print(f'Confusion Matrix:\n{conf_matrix}')
    print("\n\n")
    metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1, "roc_auc": roc_auc, "conf_matrix": conf_matrix, "best_hyperparameters": grid_search.best_params_, "best_score": grid_search.best_score_}

    # Save model and metrics 
    curr_result_path = os.path.join(specific_results_path, trait)
    os.makedirs(curr_result_path, exist_ok=True)
    with open(os.path.join(curr_result_path, f'knn_model_tuned.pkl'), 'wb') as file:
        pickle.dump(best_knn, file)
    with open(os.path.join(curr_result_path, f'perf_metrics_tuned.pkl'), 'wb') as file:
        pickle.dump(metrics, file)

Processing Extraversion
Best Hyperparameters:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 11, 'p': 2, 'weights': 'distance'}
Best Score:  0.5719636169074371
Accuracy: 0.5288461538461539
Precision: 0.5092969343197521
Recall: 0.5288461538461539
F1-Score: 0.5056383889388538
ROC AUC: [0.5897435897435899, 0.6281076066790353, 0.5702169219697748]
Confusion Matrix:
[[10 18 11]
 [ 5 75 18]
 [ 8 38 25]]



Processing Agreeableness
Best Hyperparameters:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Best Score:  0.64258962011771
Accuracy: 0.6009615384615384
Precision: 0.5987224353366086
Recall: 0.6009615384615384
F1-Score: 0.5997186080851419
ROC AUC: [0.582757296466974, 0.5827572964669738]
Confusion Matrix:
[[41 43]
 [40 84]]



Processing Conscientiousness
Best Hyperparameters:  {'algorithm': 'auto', 'leaf_size': 10, 'metric': 'minkowski', 'n_neighbors': 13, 'p': 1, 'weights': 'distance'}
Best Scor